In [96]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import display

In [97]:
df_data = pd.read_csv('data/House Price India.csv')
df_data.drop(columns=['Built Year', 'Renovation Year', 'id', 'Date', 'Lattitude', 'Longitude', 'Postal Code', 'waterfront present', 'number of views'], inplace=True)
df_data

,number of bedrooms,number of bathrooms,living area,lot area,number of floors,condition of the house,grade of the house,Area of the house(excluding basement),Area of the basement,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport,Price
0,5,2.50,3650,9050,2.0,5,10,3370,280,2880,5400,2,58,2380000
1,4,2.50,2920,4000,1.5,5,8,1910,1010,2470,4000,2,51,1400000
2,5,2.75,2910,9480,1.5,3,8,2910,0,2940,6600,1,53,1200000
3,4,2.50,3310,42998,2.0,3,9,3310,0,3350,42847,3,76,838000
4,3,2.00,2710,4500,1.5,4,8,1880,830,2060,4500,1,51,805000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14615,2,1.50,1556,20000,1.0,4,7,1556,0,2250,17286,3,76,221700
14616,3,2.00,1680,7000,1.5,4,7,1680,0,1540,7480,3,59,219200
14617,2,1.00,1070,6120,1.0,3,6,1070,0,1130,6120,2,64,209000
14618,4,1.00,1030,6621,1.0,4,6,1030,0,1420,6631,3,54,205000


In [98]:
df_data.columns

Index(['number of bedrooms', 'number of bathrooms', 'living area', 'lot area',
       'number of floors', 'condition of the house', 'grade of the house',
       'Area of the house(excluding basement)', 'Area of the basement',
       'living_area_renov', 'lot_area_renov', 'Number of schools nearby',
       'Distance from the airport', 'Price'],
      dtype='object')

In [99]:
Q1 = df_data.quantile(0.25)
Q3 = df_data.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_data = df_data[~((df_data < lower_bound) | (df_data > upper_bound)).any(axis=1)]
df_data.sort_values(by=['Price'], inplace=True, ascending=True)
df_data.reset_index(drop=True, inplace=True)
df_data

C:\Users\ACER\AppData\Local\Temp\ipykernel_16704\3277719514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data.sort_values(by=['Price'], inplace=True, ascending=True)


,number of bedrooms,number of bathrooms,living area,lot area,number of floors,condition of the house,grade of the house,Area of the house(excluding basement),Area of the basement,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport,Price
0,3,1.00,860,10426,1.0,3,6,860,0,1140,11250,3,60,82000
1,2,1.00,830,9000,1.0,3,6,830,0,1160,7680,2,75,85000
2,3,1.00,840,9480,1.0,3,6,840,0,840,9420,3,72,86500
3,3,1.00,900,4750,1.0,4,6,900,0,900,3404,2,68,89000
4,2,1.00,790,2640,1.0,3,7,790,0,1310,2064,3,76,90000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11219,4,2.50,2520,2600,2.0,5,8,1670,850,2290,3600,2,55,1130000
11220,3,2.25,2790,13791,1.0,3,8,2790,0,2720,12600,2,53,1130000
11221,5,1.75,1910,5640,2.0,4,9,1910,0,2050,5400,3,67,1130000
11222,4,2.50,2660,11200,2.0,3,9,2660,0,3290,11275,2,56,1130000


In [100]:
def categorize(value, min_val, max_val, divisions):
    range_val = max_val - min_val
    segment_size = range_val / divisions
    for i in range(divisions):
        if value < (min_val + segment_size * (i + 1)):
            return 'low' if i == 0 else 'medium' if i < divisions - 1 else 'high'
    return 'high'

df_rule = pd.DataFrame()
df_rule['number of bedrooms'] = df_data['number of bedrooms'].apply(lambda x: categorize(x, df_data['number of bedrooms'].min(), df_data['number of bedrooms'].max(), 2))
df_rule['number of bathrooms'] = df_data['number of bathrooms'].apply(lambda x: categorize(x, df_data['number of bathrooms'].min(), df_data['number of bathrooms'].max(), 2))
df_rule['living area'] = df_data['living area'].apply(lambda x: categorize(x, df_data['living area'].min(), df_data['living area'].max(), 2))
df_rule['lot area'] = df_data['lot area'].apply(lambda x: categorize(x, df_data['lot area'].min(), df_data['lot area'].max(), 3))
df_rule['number of floors'] = df_data['number of floors'].apply(lambda x: categorize(x, df_data['number of floors'].min(), df_data['number of floors'].max(), 2))
df_rule['condition of the house'] = df_data['condition of the house'].apply(lambda x: categorize(x, df_data['condition of the house'].min(), df_data['condition of the house'].max(), 2))
df_rule['grade of the house'] = df_data['grade of the house'].apply(lambda x: categorize(x, df_data['grade of the house'].min(), df_data['grade of the house'].max(), 2))
df_rule['Area of the house(excluding basement)'] = df_data['Area of the house(excluding basement)'].apply(lambda x: categorize(x, df_data['Area of the house(excluding basement)'].min(), df_data['Area of the house(excluding basement)'].max(), 3))
df_rule['Area of the basement'] = df_data['Area of the basement'].apply(lambda x: categorize(x, df_data['Area of the basement'].min(), df_data['Area of the basement'].max(), 3))
df_rule['living_area_renov'] = df_data['living_area_renov'].apply(lambda x: categorize(x, df_data['living_area_renov'].min(), df_data['living_area_renov'].max(), 3))
df_rule['lot_area_renov'] = df_data['lot_area_renov'].apply(lambda x: categorize(x, df_data['lot_area_renov'].min(), df_data['lot_area_renov'].max(), 3))
df_rule['Number of schools nearby'] = df_data['Number of schools nearby'].apply(lambda x: categorize(x, df_data['Number of schools nearby'].min(), df_data['Number of schools nearby'].max(), 2))
df_rule['Distance from the airport'] = df_data['Distance from the airport'].apply(lambda x: categorize(x, df_data['Distance from the airport'].min(), df_data['Distance from the airport'].max(), 2))
df_rule['Price'] = df_data['Price'].apply(lambda x: categorize(x, df_data['Price'].min(), df_data['Price'].max(), 3))

In [101]:
df_rule.drop_duplicates(inplace=True, subset=['number of bedrooms', 'number of bathrooms', 'living area', 'lot area',
       'number of floors', 'condition of the house', 'grade of the house',
       'Area of the house(excluding basement)', 'Area of the basement',
       'living_area_renov', 'lot_area_renov', 'Number of schools nearby',
       'Distance from the airport'])
df_rule.reset_index(drop=True,inplace=True)

In [102]:
df_rule

,number of bedrooms,number of bathrooms,living area,lot area,number of floors,condition of the house,grade of the house,Area of the house(excluding basement),Area of the basement,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport,Price
0,low,low,low,medium,low,low,low,low,low,low,medium,high,low,low
1,low,low,low,medium,low,low,low,low,low,low,medium,high,high,low
2,low,low,low,low,low,high,low,low,low,low,low,high,high,low
3,low,low,low,low,low,low,low,low,low,low,low,high,high,low
4,low,low,low,low,low,high,low,low,low,low,low,high,low,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,high,high,high,low,low,high,high,medium,medium,low,low,low,low,high
2528,high,low,high,low,low,high,high,medium,low,medium,medium,high,high,high
2529,high,high,high,high,low,high,high,medium,high,high,high,high,low,high
2530,low,high,high,high,low,low,high,high,low,high,high,high,low,high


In [103]:
def rule_to_index(df):
    for column in df.columns:
        if df[column].value_counts().shape[0] > 2:
            df[column] = df[column].astype('category').cat.codes
        else:
            df[column] = df[column].apply(lambda x: 1 if x == 'high' else 0)
    return df

In [104]:
df_rule = rule_to_index(df_rule)

In [105]:
df_rule

,number of bedrooms,number of bathrooms,living area,lot area,number of floors,condition of the house,grade of the house,Area of the house(excluding basement),Area of the basement,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport,Price
0,0,0,0,2,0,0,0,1,1,1,2,1,0,1
1,0,0,0,2,0,0,0,1,1,1,2,1,1,1
2,0,0,0,1,0,1,0,1,1,1,1,1,1,1
3,0,0,0,1,0,0,0,1,1,1,1,1,1,1
4,0,0,0,1,0,1,0,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,1,1,1,1,0,1,1,2,2,1,1,0,0,0
2528,1,0,1,1,0,1,1,2,1,2,2,1,1,0
2529,1,1,1,0,0,1,1,2,0,0,0,1,0,0
2530,0,1,1,0,0,0,1,0,1,0,0,1,0,0


In [106]:
def down_function(x, a, b):
    result = (b - x) / (b - a)
    result = round(result, 2)
    return max(result, 0)

def up_function(x, a, b):
    result = (x - a) / (b - a)
    result = round(result, 2)
    return max(result, 0)

def triangle_function(x, a, b, c):
    if x <= a or x >= c:
        return 0
    if a < x <= b:
        return up_function(x, a, b)
    if b < x < c:
        return down_function(x, b, c)

In [107]:
def bedroom_membership(x):
    low = down_function(x, df_data['number of bedrooms'].min(), df_data['number of bedrooms'].max())
    medium = up_function(x, df_data['number of bedrooms'].min(), df_data['number of bedrooms'].max())
    return low, medium

def bathroom_membership(x):
    low = down_function(x, df_data['number of bathrooms'].min(), df_data['number of bathrooms'].max())
    medium = up_function(x, df_data['number of bathrooms'].min(), df_data['number of bathrooms'].max())
    return low, medium

def living_area_membership(x):
    low = down_function(x, df_data['living area'].min(), df_data['living area'].max())
    medium = up_function(x, df_data['living area'].min(), df_data['living area'].max())
    return low, medium

def lot_area_membership(x):
    low = down_function(x, df_data['lot area'].min(), df_data['lot area'].max() // 2)
    medium = triangle_function(x, df_data['lot area'].min(), df_data['lot area'].max() // 2, df_data['lot area'].max())
    high = up_function(x, df_data['lot area'].max() // 2, df_data['lot area'].max())
    return low, medium, high

def number_of_floors_membership(x):
    low = down_function(x, df_data['number of floors'].min(), df_data['number of floors'].max())
    medium = up_function(x, df_data['number of floors'].min(), df_data['number of floors'].max())
    return low, medium

def condition_of_the_house_membership(x):
    low = down_function(x, df_data['condition of the house'].min(), df_data['condition of the house'].max())
    medium = up_function(x, df_data['condition of the house'].min(), df_data['condition of the house'].max())
    return low, medium

def grade_of_the_house_membership(x):
    low = down_function(x, df_data['grade of the house'].min(), df_data['grade of the house'].max())
    medium = up_function(x, df_data['grade of the house'].min(), df_data['grade of the house'].max())
    return low, medium

def area_of_the_house_membership(x):
    low = down_function(x, df_data['Area of the house(excluding basement)'].min(), df_data['Area of the house(excluding basement)'].max() // 2)
    medium = triangle_function(x, df_data['Area of the house(excluding basement)'].min(), df_data['Area of the house(excluding basement)'].max() // 2, df_data['Area of the house(excluding basement)'].max())
    high = up_function(x, df_data['Area of the house(excluding basement)'].max() // 2, df_data['Area of the house(excluding basement)'].max())
    return low, medium, high

def area_of_the_basement_membership(x):
    low = down_function(x, df_data['Area of the basement'].min(), df_data['Area of the basement'].max() // 2)
    medium = triangle_function(x, df_data['Area of the basement'].min(), df_data['Area of the basement'].max() // 2, df_data['Area of the basement'].max())
    high = up_function(x, df_data['Area of the basement'].max() // 2, df_data['Area of the basement'].max())
    return low, medium, high

def living_area_renov_membership(x):
    low = down_function(x, df_data['living_area_renov'].min(), df_data['living_area_renov'].max() // 2)
    medium = triangle_function(x, df_data['living_area_renov'].min(), df_data['living_area_renov'].max() // 2, df_data['living_area_renov'].max())
    high = up_function(x, df_data['living_area_renov'].max() // 2, df_data['living_area_renov'].max())
    return low, medium, high

def lot_area_renov_membership(x):
    low = down_function(x, df_data['lot_area_renov'].min(), df_data['lot_area_renov'].max() // 2)
    medium = triangle_function(x, df_data['lot_area_renov'].min(), df_data['lot_area_renov'].max() // 2, df_data['lot_area_renov'].max())
    high = up_function(x, df_data['lot_area_renov'].max() // 2, df_data['lot_area_renov'].max())
    return low, medium, high

def number_of_schools_nearby_membership(x):
    low = down_function(x, df_data['Number of schools nearby'].min(), df_data['Number of schools nearby'].max())
    medium = up_function(x, df_data['Number of schools nearby'].min(), df_data['Number of schools nearby'].max())
    return low, medium

def distance_from_the_airport_membership(x):
    low = down_function(x, df_data['Distance from the airport'].min(), df_data['Distance from the airport'].max())
    medium = up_function(x, df_data['Distance from the airport'].min(), df_data['Distance from the airport'].max())
    return low, medium

In [108]:
def calculate_membership(df_data):
    membership_functions = {
        'number of bedrooms': bedroom_membership,
        'number of bathrooms': bathroom_membership,
        'living area': living_area_membership,
        'lot area': lot_area_membership,
        'number of floors': number_of_floors_membership,
        'condition of the house': condition_of_the_house_membership,
        'grade of the house': grade_of_the_house_membership,
        'Area of the house(excluding basement)': area_of_the_house_membership,
        'Area of the basement': area_of_the_basement_membership,
        'living_area_renov': living_area_renov_membership,
        'lot_area_renov': lot_area_renov_membership,
        'Number of schools nearby': number_of_schools_nearby_membership,
        'Distance from the airport': distance_from_the_airport_membership
    }
    df_result = pd.DataFrame()
    for column, membership_function in membership_functions.items():
        df_result[column] = df_data[column].apply(lambda x: membership_function(x))
    return df_result

In [109]:
df_membership = calculate_membership(df_data.drop(columns=['Price']))

In [110]:
df_membership.reset_index(drop=True, inplace=True)

In [111]:
display(df_membership)
display(df_rule.drop(columns=['Price']))

,number of bedrooms,number of bathrooms,living area,lot area,number of floors,condition of the house,grade of the house,Area of the house(excluding basement),Area of the basement,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport
0,"(0.67, 0.33)","(0.91, 0.09)","(0.92, 0.08)","(0, 0.93, 0.07)","(1.0, 0.0)","(0.67, 0.33)","(1.0, 0.0)","(0.73, 0.27, 0)","(1.0, 0, 0)","(0.58, 0.42, 0)","(0, 0.73, 0.27)","(0.0, 1.0)","(0.67, 0.33)"
1,"(1.0, 0.0)","(0.91, 0.09)","(0.93, 0.07)","(0.08, 0.92, 0)","(1.0, 0.0)","(0.67, 0.33)","(1.0, 0.0)","(0.75, 0.25, 0)","(1.0, 0, 0)","(0.56, 0.44, 0)","(0.14, 0.86, 0)","(0.5, 0.5)","(0.17, 0.83)"
2,"(0.67, 0.33)","(0.91, 0.09)","(0.92, 0.08)","(0.03, 0.97, 0)","(1.0, 0.0)","(0.67, 0.33)","(1.0, 0.0)","(0.75, 0.25, 0)","(1.0, 0, 0)","(0.82, 0.18, 0)","(0, 0.93, 0.07)","(0.0, 1.0)","(0.27, 0.73)"
3,"(0.67, 0.33)","(0.91, 0.09)","(0.91, 0.09)","(0.54, 0.46, 0)","(1.0, 0.0)","(0.33, 0.67)","(1.0, 0.0)","(0.7, 0.3, 0)","(1.0, 0, 0)","(0.77, 0.23, 0)","(0.66, 0.34, 0)","(0.5, 0.5)","(0.4, 0.6)"
4,"(1.0, 0.0)","(0.91, 0.09)","(0.94, 0.06)","(0.77, 0.23, 0)","(1.0, 0.0)","(0.67, 0.33)","(0.67, 0.33)","(0.78, 0.22, 0)","(1.0, 0, 0)","(0.44, 0.56, 0)","(0.83, 0.17, 0)","(0.0, 1.0)","(0.13, 0.87)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11219,"(0.33, 0.67)","(0.36, 0.64)","(0.46, 0.54)","(0.77, 0.23, 0)","(0.6, 0.4)","(0.0, 1.0)","(0.33, 0.67)","(0.16, 0.84, 0)","(0, 0.83, 0.17)","(0, 0.76, 0.24)","(0.64, 0.36, 0)","(0.5, 0.5)","(0.83, 0.17)"
11220,"(0.67, 0.33)","(0.45, 0.55)","(0.39, 0.61)","(0, 0.58, 0.42)","(1.0, 0.0)","(0.67, 0.33)","(0.33, 0.67)","(0, 0.53, 0.47)","(1.0, 0, 0)","(0, 0.53, 0.47)","(0, 0.57, 0.43)","(0.5, 0.5)","(0.9, 0.1)"
11221,"(0.0, 1.0)","(0.64, 0.36)","(0.63, 0.37)","(0.44, 0.56, 0)","(0.6, 0.4)","(0.33, 0.67)","(0.0, 1.0)","(0, 0.99, 0.01)","(1.0, 0, 0)","(0, 0.89, 0.11)","(0.42, 0.58, 0)","(0.0, 1.0)","(0.43, 0.57)"
11222,"(0.33, 0.67)","(0.36, 0.64)","(0.42, 0.58)","(0, 0.85, 0.15)","(0.6, 0.4)","(0.67, 0.33)","(0.0, 1.0)","(0, 0.6, 0.4)","(1.0, 0, 0)","(0, 0.22, 0.78)","(0, 0.72, 0.28)","(0.5, 0.5)","(0.8, 0.2)"


,number of bedrooms,number of bathrooms,living area,lot area,number of floors,condition of the house,grade of the house,Area of the house(excluding basement),Area of the basement,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport
0,0,0,0,2,0,0,0,1,1,1,2,1,0
1,0,0,0,2,0,0,0,1,1,1,2,1,1
2,0,0,0,1,0,1,0,1,1,1,1,1,1
3,0,0,0,1,0,0,0,1,1,1,1,1,1
4,0,0,0,1,0,1,0,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,1,1,1,1,0,1,1,2,2,1,1,0,0
2528,1,0,1,1,0,1,1,2,1,2,2,1,1
2529,1,1,1,0,0,1,1,2,0,0,0,1,0
2530,0,1,1,0,0,0,1,0,1,0,0,1,0


In [112]:
def aggregate(df_membership, df_rule):
    alpha_predicate = np.zeros((df_membership.shape[0], df_rule.shape[0]))
    for i, row_member in df_membership.iterrows():
        for j, row_rule in df_rule.iterrows():
            temp = []
            for cell_member, cell_rule in zip(row_member, row_rule):
                temp.append(cell_member[cell_rule])
            alpha_predicate[i, j] = min(temp)
    return alpha_predicate

In [113]:
df_aggregated = aggregate(df_membership, df_rule.drop(columns=['Price']))
df_aggregated.shape

(11224, 2532)

In [114]:
def inv_up_function(alpha, a, b):
    return alpha*(b-a) + a

def inv_down_function(alpha, a, b):
    return b - alpha*(b-a)

def calculate_price(df_aggregated, prices):
    list_price = []
    for i in range(df_aggregated.shape[0]):
        dict_price = {
            'a_pred': [],
            'z': []
         }
        for j, price in enumerate(prices):
            if price == 0:
                dict_price['a_pred'].append(df_aggregated[i, j])
                dict_price['z'].append(inv_down_function(df_aggregated[i, j], df_data['Price'].min(), df_data['Price'].max()//2))   
            elif price == 1:
                dict_price['a_pred'].append(df_aggregated[i, j])
                dict_price['z'].append(inv_up_function(df_aggregated[i, j], df_data['Price'].min(), df_data['Price'].max()//2)) 
                dict_price['a_pred'].append(df_aggregated[i, j])
                dict_price['z'].append(inv_down_function(df_aggregated[i, j], df_data['Price'].max()//2, df_data['Price'].max()))
            elif price == 2:
                dict_price['a_pred'].append(df_aggregated[i, j])
                dict_price['z'].append(inv_up_function(df_aggregated[i, j], df_data['Price'].max()//2, df_data['Price'].max()))
        list_price.append(dict_price)
    return list_price

In [115]:
prices = calculate_price(df_aggregated, df_rule['Price']) 

In [128]:
def defuzzification(prices):
    result = []
    for price in prices:
        a_pred = price['a_pred']
        z = price['z']
        numerator = 0
        denominator = 0
        for a, z in zip(a_pred, z):
            numerator += a*z
            denominator += a
        result.append(numerator/denominator if denominator != 0 else 0)
    return result

In [129]:
predicted_price = defuzzification(prices)

In [130]:
for y, y_hat in zip(df_data['Price'], predicted_price):
    print(f'Actual: {y}, Predicted: {y_hat}')

Actual: 82000, Predicted: 628146.4383561645
Actual: 85000, Predicted: 640426.846846847
Actual: 86500, Predicted: 617404.1554959787
Actual: 89000, Predicted: 625684.1246290801
Actual: 90000, Predicted: 621562.6451612902
Actual: 90000, Predicted: 621035.9275053305
Actual: 95000, Predicted: 642994.1573033709
Actual: 96500, Predicted: 611051.8404907973
Actual: 100000, Predicted: 634901.4285714286
Actual: 100000, Predicted: 686994.4444444445
Actual: 100000, Predicted: 604218.8586956522
Actual: 104950, Predicted: 750928.75
Actual: 105000, Predicted: 611039.913644214
Actual: 107000, Predicted: 612442.9032258063
Actual: 107000, Predicted: 638124.7605633803
Actual: 109500, Predicted: 645745.1515151515
Actual: 110000, Predicted: 629874.0111420614
Actual: 110700, Predicted: 614291.8367346937
Actual: 114000, Predicted: 623211.6969696969
Actual: 114975, Predicted: 625015.0793650794
Actual: 115000, Predicted: 657496.530612245
Actual: 118125, Predicted: 618045.2173913044
Actual: 119500, Predicted: 63

In [131]:
error = []
for y, y_hat in zip(df_data['Price'], predicted_price):
    if y_hat:
        error.append(abs(y - y_hat))
print(f'Mean relative error: {np.mean(error)}')

Mean relative error: 226163.3468892862
